**결측치처리방식변경**
+ 기존 mode 값으로 바꾸는 대신, 수치형은 -1, string type은 'null' 로 바꿈
+ 결과값 개선됨

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
path = '/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)'
os.chdir(path)
print(os.getcwd())

/content/drive/MyDrive/github/Dacon/항공편지연예측AI경진대회(2023.04.03-2023.05.08)


In [2]:
import pandas as pd
import numpy as np
import random
import os
import gc

from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

### Data Load

In [5]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

print(train.shape)
print(train.Delay.value_counts())


(1000000, 19)
Not_Delayed    210001
Delayed         45000
Name: Delay, dtype: int64


In [ ]:
train.isnull().sum()

ID                               0
Month                            0
Day_of_Month                     0
Estimated_Departure_Time    109019
Estimated_Arrival_Time      109040
Cancelled                        0
Diverted                         0
Origin_Airport                   0
Origin_Airport_ID                0
Origin_State                109015
Destination_Airport              0
Destination_Airport_ID           0
Destination_State           109079
Distance                         0
Airline                     108920
Carrier_Code(IATA)          108990
Carrier_ID(DOT)             108997
Tail_Number                      0
Delay                       744999
dtype: int64

### 데이터 전처리
+ 결측치 처리

In [5]:
# 레이블(Delay)을 제외한 결측값이 존재하는 변수들을  -1 value로 바꾸고, lable을 가지도록 합니다.
NaN_col = ['Origin_State','Destination_State','Airline','Estimated_Departure_Time', 'Estimated_Arrival_Time','Carrier_Code(IATA)','Carrier_ID(DOT)']

for col in NaN_col:
    mode = train[col].mode()[0]
    if type(mode) == str:
        train[col] = train[col].fillna('null')
    else:
        train[col] = train[col].fillna(-1)
    
    if col in test.columns:
        if type(mode) == str:
            test[col] = test[col].fillna('null')
        else:
            test[col] = test[col].fillna(-1)
print('Done.')

Done.


In [6]:
# 질적 변수들을 수치화합니다
qual_col = ['Origin_Airport', 'Origin_State', 'Destination_Airport', 'Destination_State', 'Airline', 'Carrier_Code(IATA)', 'Tail_Number']

for i in qual_col:
    le = LabelEncoder()
    le=le.fit(train[i])
    train[i]=le.transform(train[i])
    
    for label in np.unique(test[i]):
        if label not in le.classes_: # train에 없는 label인 경우
            le.classes_ = np.append(le.classes_, label)
    test[i]=le.transform(test[i])
print('Done.')

Done.


In [ ]:
train.head()

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay
0,TRAIN_000000,4,15,600.0,1900.0,0,0,252,13851,34,159,12191,42,419.0,23,10,19393.0,4319,None
1,TRAIN_000001,8,15,740.0,1024.0,0,0,256,13930,11,331,14869,45,1250.0,22,8,20304.0,310,None
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,74,11057,31,204,12953,30,544.0,3,0,19805.0,140,None
3,TRAIN_000003,7,10,905.0,1735.0,0,0,195,12892,4,119,11618,28,2454.0,26,8,19393.0,3021,None
4,TRAIN_000004,1,11,900.0,1019.0,0,0,322,14771,4,7,10157,4,250.0,22,8,20304.0,556,None


In [7]:
# 레이블이 없는 데이터들을 제거합니다
train = train.dropna()
print(train.shape)

# Delay -> 1, Not_Delayed -> 0
column_number = {}
for i, column in enumerate(sample_submission.columns):
    column_number[column] = i
    
def to_number(x, dic):
    return dic[x]

train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))
print('Done.')

train.head()

(255001, 19)
Done.


<ipython-input-7-3f0141e48b31>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.loc[:, 'Delay_num'] = train['Delay'].apply(lambda x: to_number(x, column_number))


,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport,Origin_Airport_ID,Origin_State,Destination_Airport,Destination_Airport_ID,Destination_State,Distance,Airline,Carrier_Code(IATA),Carrier_ID(DOT),Tail_Number,Delay,Delay_num
5,TRAIN_000005,4,13,1545.0,-1.0,0,0,119,11618,52,93,11278,47,199.0,21,8,20452.0,3435,Not_Delayed,0
6,TRAIN_000006,1,20,1742.0,1903.0,0,0,119,11618,28,47,10721,19,200.0,26,8,-1.0,3495,Not_Delayed,0
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,59,10821,52,74,11057,31,361.0,23,10,19393.0,4083,Not_Delayed,0
10,TRAIN_000010,8,13,1730.0,1844.0,0,0,93,11278,47,277,14122,36,204.0,21,0,-1.0,241,Delayed,1
12,TRAIN_000012,1,12,1015.0,1145.0,0,0,72,11042,33,94,11292,5,1201.0,23,10,-1.0,5171,Not_Delayed,0


In [8]:
# train & test 데이터 생성
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

### train_test_split

In [ ]:
from sklearn.model_selection  import train_test_split
train_x, val_x, train_y, val_y = train_test_split(train.drop(columns=['ID', 'Delay', 'Delay_num']), train['Delay_num'], test_size=0.2, random_state=42)

print(train_x.shape, val_x.shape)
print(train_y.value_counts())
print(val_y.value_counts())

(204000, 17) (51001, 17)
0    168109
1     35891
Name: Delay_num, dtype: int64
0    41892
1     9109
Name: Delay_num, dtype: int64


### 여러 모델 적합
Extra Trees Classifier	
Random Forest Classifier	
Light Gradient Boosting Machine
Decision Tree Classifier	
Gradient Boosting Classifier	
Ada Boost Classifier	
Logistic Regression

In [ ]:
# SMOTE 없이 적합
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


# 모델 정의
models = {
    "Extra Trees Classifier": ExtraTreesClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "Light Gradient Boosting Machine": LGBMClassifier(random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42),
    "Ada Boost Classifier": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
}

# 각 모델의 성능을 비교
for name, model in models.items():
    model.fit(train_x, train_y)
    y_pred = model.predict_proba(val_x)
    loss = log_loss(val_y, y_pred)
    print(f"{name}: Log Loss = {loss:.4f}")

Extra Trees Classifier: Log Loss = 0.4799
Random Forest Classifier: Log Loss = 0.4646
Light Gradient Boosting Machine: Log Loss = 0.4433
Decision Tree Classifier: Log Loss = 10.5274
Gradient Boosting Classifier: Log Loss = 0.4480
Ada Boost Classifier: Log Loss = 0.6823
Logistic Regression: Log Loss = 0.4621


In [ ]:
# SMOTE 이용해서 적합
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from imblearn.over_sampling import SMOTE

from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from lightgbm import LGBMClassifier


# SMOTE로 데이터 불균형 완화
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(train_x, train_y)


# 모델 정의
models = {
    "Extra Trees Classifier": ExtraTreesClassifier(random_state=42),
    "Random Forest Classifier": RandomForestClassifier(random_state=42),
    "Light Gradient Boosting Machine": LGBMClassifier(random_state=42),
    "Decision Tree Classifier": DecisionTreeClassifier(random_state=42),
    "Gradient Boosting Classifier": GradientBoostingClassifier(random_state=42),
    "Ada Boost Classifier": AdaBoostClassifier(random_state=42),
    "Logistic Regression": LogisticRegression(random_state=42),
}

# 각 모델의 성능을 비교
for name, model in models.items():
    model.fit(X_train_smote, y_train_smote)
    y_pred = model.predict_proba(val_x)
    loss = log_loss(val_y, y_pred)
    print(f"{name}: Log Loss = {loss:.4f}")

Extra Trees Classifier: Log Loss = 0.5113
Random Forest Classifier: Log Loss = 0.5092
Light Gradient Boosting Machine: Log Loss = 0.5127
Decision Tree Classifier: Log Loss = 11.6489
Gradient Boosting Classifier: Log Loss = 0.6037
Ada Boost Classifier: Log Loss = 0.6896
Logistic Regression: Log Loss = 0.6813


### 모델적합
+ 최저의 Loss를 갖는 모델 적합
    + lgbm 이용

In [10]:
# grid search 1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [3, 5, 7],
    "learning_rate": [0.01, 0.1, 0.2],
    "num_leaves": [7, 15, 31],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search.best_params_}")
print(f"최고의 점수: {-1 * grid_search.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm = grid_search.best_estimator_


Fitting 5 folds for each of 81 candidates, totalling 405 fits
최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'num_leaves': 31}
최고의 점수: 0.4395


In [14]:
# grid search2 from grid search1
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from lightgbm import LGBMClassifier

# 전체 데이터 이용
train_x = train.drop(columns=['ID', 'Delay', 'Delay_num'])
train_y = train['Delay_num']
test_x = test.drop(columns=['ID'])

# 모델 정의
lgbm = LGBMClassifier(random_state=42)

# 그리드 서치를 위한 파라미터 그리드 정의
param_grid2 = {
    "n_estimators": [150, 200, 250, 300],
    "max_depth": [5, 7, 9],
    "learning_rate": [0.05, 0.075, 0.1, 0.125],
    "num_leaves": [20, 30, 40],
}

# 그리드 서치를 위한 GridSearchCV 객체 생성
grid_search2 = GridSearchCV(
    estimator=lgbm,
    param_grid=param_grid2,
    scoring="neg_log_loss",
    cv=5,
    verbose=1,
    n_jobs=-1,
)

# 그리드 서치를 수행
grid_search2.fit(train_x, train_y)

# 최적의 파라미터와 최고의 점수 출력
print(f"최적의 파라미터: {grid_search2.best_params_}")
print(f"최고의 점수: {-1 * grid_search2.best_score_:.4f}")

# 최적의 파라미터를 사용한 최종 모델
best_lgbm2 = grid_search2.best_estimator_


Fitting 5 folds for each of 144 candidates, totalling 720 fits
최적의 파라미터: {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 300, 'num_leaves': 30}
최고의 점수: 0.4391


### 예측

In [11]:
y_pred = best_lgbm.predict_proba(test_x)

In [15]:
y_pred2 = best_lgbm2.predict_proba(test_x)

### 제출

In [12]:
submission = pd.DataFrame(data=y_pred, columns=sample_submission.columns, index=sample_submission.index)
submission.to_csv('submission_lgbm.csv', index=True)

In [16]:
submission2 = pd.DataFrame(data=y_pred2, columns=sample_submission.columns, index=sample_submission.index)
submission2.to_csv('submission_lgbm2.csv', index=True)